# Human-in-the-Loop Workflow gamit ang Microsoft Agent Framework

## 🎯 Mga Layunin sa Pag-aaral

Sa notebook na ito, matututunan mo kung paano ipatupad ang **human-in-the-loop** workflows gamit ang `RequestInfoExecutor` ng Microsoft Agent Framework. Ang makapangyarihang pattern na ito ay nagbibigay-daan sa pag-pause ng AI workflows upang makakuha ng input mula sa tao, ginagawa ang iyong mga agent na interactive at binibigyan ang mga tao ng kontrol sa mahahalagang desisyon.

## 🔄 Ano ang Human-in-the-Loop?

Ang **Human-in-the-loop (HITL)** ay isang disenyo kung saan ang mga AI agent ay humihinto sa pagproseso upang humingi ng input mula sa tao bago magpatuloy. Mahalaga ito para sa:

- ✅ **Mahahalagang desisyon** - Kunin ang pag-apruba ng tao bago gumawa ng mahalagang aksyon
- ✅ **Mga hindi malinaw na sitwasyon** - Hayaan ang tao na magbigay-linaw kapag hindi sigurado ang AI
- ✅ **Mga kagustuhan ng user** - Magtanong sa mga user upang pumili sa pagitan ng maraming opsyon
- ✅ **Pagsunod at kaligtasan** - Siguraduhin ang pangangasiwa ng tao para sa mga reguladong operasyon
- ✅ **Mga interactive na karanasan** - Gumawa ng mga conversational agent na tumutugon sa input ng user

## 🏗️ Paano Ito Gumagana sa Microsoft Agent Framework

Ang framework ay nagbibigay ng tatlong pangunahing bahagi para sa HITL:

1. **`RequestInfoExecutor`** - Isang espesyal na executor na humihinto sa workflow at naglalabas ng `RequestInfoEvent`
2. **`RequestInfoMessage`** - Base class para sa mga typed request payloads na ipinapadala sa tao
3. **`RequestResponse`** - Nag-uugnay ng mga tugon ng tao sa orihinal na mga request gamit ang `request_id`

**Pattern ng Workflow:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Ang Ating Halimbawa: Pag-book ng Hotel na may Kumpirmasyon ng User

Magpapatuloy tayo sa conditional workflow sa pamamagitan ng pagdaragdag ng kumpirmasyon mula sa tao **bago** magmungkahi ng mga alternatibong destinasyon:

1. Humihiling ang user ng destinasyon (hal., "Paris")
2. Sinusuri ng `availability_agent` kung may mga available na kwarto
3. **Kung walang kwarto** → Tatanungin ng `confirmation_agent` ang "Gusto mo bang makita ang mga alternatibo?"
4. Ang workflow ay **humihinto** gamit ang `RequestInfoExecutor`
5. **Sumasagot ang tao** ng "oo" o "hindi" sa pamamagitan ng console input
6. Ang `decision_manager` ay nagruruta batay sa tugon:
   - **Oo** → Ipakita ang mga alternatibong destinasyon
   - **Hindi** → Kanselahin ang kahilingan sa pag-book
7. Ipakita ang huling resulta

Ipinapakita nito kung paano bigyan ang mga user ng kontrol sa mga mungkahi ng agent!

---

Simulan na natin! 🚀


## Hakbang 1: I-import ang Mga Kinakailangang Library

Ini-import natin ang mga karaniwang bahagi ng Agent Framework kasama ang **mga klase na partikular sa human-in-the-loop**:
- `RequestInfoExecutor` - Executor na humihinto sa workflow para sa input ng tao
- `RequestInfoEvent` - Event na inilalabas kapag kinakailangan ang input ng tao
- `RequestInfoMessage` - Pangunahing klase para sa mga uri ng request payload
- `RequestResponse` - Nag-uugnay ng mga sagot ng tao sa mga request
- `WorkflowOutputEvent` - Event para sa pag-detect ng mga output ng workflow


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Hakbang 2: Tukuyin ang mga Modelong Pydantic para sa Nakabalangkas na Output

Ang mga modelong ito ang nagtatakda ng **schema** na ibabalik ng mga ahente. Pinapanatili natin ang lahat ng modelo mula sa conditional workflow at nagdaragdag ng:

**Bago para sa Human-in-the-Loop:**
- `HumanFeedbackRequest` - Subclass ng `RequestInfoMessage` na tumutukoy sa payload ng kahilingan na ipinapadala sa mga tao
  - Naglalaman ng `prompt` (tanong na itatanong) at `destination` (konteksto tungkol sa hindi available na lungsod)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Hakbang 3: Gumawa ng Hotel Booking Tool

Parehong tool mula sa conditional workflow - sinusuri kung may mga available na kwarto sa destinasyon.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Hakbang 4: Tukuyin ang Mga Function ng Kondisyon para sa Routing

Kailangan natin ng **apat na function ng kondisyon** para sa ating human-in-the-loop workflow:

**Mula sa conditional workflow:**
1. `has_availability_condition` - Nagre-route kapag MAY available na mga hotel
2. `no_availability_condition` - Nagre-route kapag WALANG available na mga hotel

**Bago para sa human-in-the-loop:**
3. `user_wants_alternatives_condition` - Nagre-route kapag sinabi ng user na "oo" sa mga alternatibo
4. `user_declines_alternatives_condition` - Nagre-route kapag sinabi ng user na "hindi" sa mga alternatibo


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Hakbang 5: Gumawa ng Decision Manager Executor

Ito ang **puso ng human-in-the-loop pattern**! Ang `DecisionManager` ay isang custom na `Executor` na:

1. **Tumanggap ng feedback mula sa tao** gamit ang mga `RequestResponse` na object
2. **Pinoproseso ang desisyon ng user** (oo/hindi)
3. **Inaayos ang workflow** sa pamamagitan ng pagpapadala ng mga mensahe sa tamang mga ahente

Mga pangunahing tampok:
- Gumagamit ng `@handler` na decorator upang gawing mga hakbang sa workflow ang mga method
- Tumanggap ng `RequestResponse[HumanFeedbackRequest, str]` na naglalaman ng parehong orihinal na request at sagot ng user
- Nagbibigay ng simpleng mga mensahe na "oo" o "hindi" na nagti-trigger sa ating mga condition function


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Hakbang 6: Gumawa ng Custom Display Executor

Parehong display executor mula sa conditional workflow - nagbibigay ng huling resulta bilang output ng workflow.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Hakbang 7: I-load ang mga Environment Variables

I-configure ang LLM client (GitHub Models, Azure OpenAI, o OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Hakbang 8: Gumawa ng AI Agents at Executors

Gumagawa tayo ng **anim na bahagi ng workflow**:

**Mga Ahente (nakabalot sa AgentExecutor):**
1. **availability_agent** - Sinusuri ang availability ng hotel gamit ang tool
2. **confirmation_agent** - 🆕 Naghahanda ng kahilingan para sa kumpirmasyon mula sa tao
3. **alternative_agent** - Nagmumungkahi ng alternatibong mga lungsod (kapag sinabi ng user na oo)
4. **booking_agent** - Hinihikayat ang pag-book (kapag may available na mga kwarto)
5. **cancellation_agent** - 🆕 Humahawak ng mensahe ng pagkansela (kapag sinabi ng user na hindi)

**Mga Espesyal na Executor:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` na pansamantalang humihinto sa workflow para sa input mula sa tao
7. **decision_manager** - 🆕 Custom executor na nagruruta batay sa tugon ng tao (naidefine na sa itaas)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Hakbang 9: Bumuo ng Workflow na may Human-in-the-Loop

Ngayon, gagawin natin ang workflow graph na may **conditional routing** kabilang ang human-in-the-loop na landas:

**Struktura ng Workflow:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Mahahalagang Koneksyon:**
- `availability_agent → confirmation_agent` (kapag walang kwarto)
- `confirmation_agent → prepare_human_request` (uri ng transformasyon)
- `prepare_human_request → request_info_executor` (pansamantalang hintuan para sa tao)
- `request_info_executor → decision_manager` (palaging - nagbibigay ng RequestResponse)
- `decision_manager → alternative_agent` (kapag sinabi ng user na "oo")
- `decision_manager → cancellation_agent` (kapag sinabi ng user na "hindi")
- `availability_agent → booking_agent` (kapag may available na kwarto)
- Lahat ng landas ay nagtatapos sa `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Hakbang 10: Patakbuhin ang Test Case 1 - Lungsod na WALANG Availability (Paris na may Kumpirmasyon ng Tao)

Ipinapakita ng test na ito ang **buong cycle na may tao sa loop**:

1. Humiling ng hotel sa Paris
2. Sinusuri ng availability_agent → Walang mga kwarto
3. Gumagawa ang confirmation_agent ng tanong para sa tao
4. Ang request_info_executor ay **pinipigil ang workflow** at naglalabas ng `RequestInfoEvent`
5. **Natuklasan ng Application ang event at hinihikayat ang user sa console**
6. Nagta-type ang user ng "yes" o "no"
7. Ipinapadala ng application ang sagot gamit ang `send_responses_streaming()`
8. Ang decision_manager ay nagruruta batay sa sagot
9. Ipinapakita ang huling resulta

**Pangunahing Pattern:**
- Gamitin ang `workflow.run_stream()` para sa unang iteration
- Gamitin ang `workflow.send_responses_streaming(pending_responses)` para sa mga susunod na iteration
- Makinig sa `RequestInfoEvent` upang matukoy kung kailan kailangan ang input ng tao
- Makinig sa `WorkflowOutputEvent` upang makuha ang huling resulta


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Hakbang 11: Patakbuhin ang Test Case 2 - Lungsod NA May Availability (Stockholm - Walang Kailangan na Input mula sa Tao)

Ipinapakita ng test na ito ang **direktang proseso** kapag may mga available na kuwarto:

1. Humiling ng hotel sa Stockholm  
2. availability_agent nag-check → May mga available na kuwarto ✅  
3. booking_agent nagmumungkahi ng pag-book  
4. display_result nagpapakita ng kumpirmasyon  
5. **Walang kinakailangang input mula sa tao!**

Ang daloy ng trabaho ay direktang dumadaan sa proseso nang walang interbensyon ng tao kapag may mga available na kuwarto.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Mga Pangunahing Puntos at Mga Pinakamahusay na Praktika sa Human-in-the-Loop

### ✅ Mga Natutunan Mo:

#### 1. **RequestInfoExecutor Pattern**
Ang human-in-the-loop pattern sa Microsoft Agent Framework ay gumagamit ng tatlong pangunahing bahagi:
- `RequestInfoExecutor` - Pinipigil ang workflow at naglalabas ng mga event
- `RequestInfoMessage` - Base class para sa mga typed request payloads (i-subclass ito!)
- `RequestResponse` - Nagkokonekta ng mga sagot ng tao sa orihinal na mga request

**Mahalagang Pag-unawa:**
- Ang `RequestInfoExecutor` ay HINDI nangongolekta ng input mismo - pinipigil lamang nito ang workflow
- Ang iyong application code ang dapat makinig sa `RequestInfoEvent` at mangolekta ng input
- Dapat mong tawagan ang `send_responses_streaming()` gamit ang isang dict na nagmamapa sa `request_id` sa sagot ng user

#### 2. **Streaming Execution Pattern**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Event-Driven Architecture**
Makinig sa mga partikular na event upang kontrolin ang workflow:
- `RequestInfoEvent` - Kailangan ng input ng tao (workflow ay pinigil)
- `WorkflowOutputEvent` - Ang huling resulta ay magagamit na (workflow ay tapos na)
- `WorkflowStatusEvent` - Mga pagbabago sa estado (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, atbp.)

#### 4. **Custom Executors gamit ang @handler**
Ang `DecisionManager` ay nagpapakita kung paano gumawa ng mga executor na:
- Gumagamit ng `@handler` decorator upang i-expose ang mga method bilang mga hakbang sa workflow
- Tumanggap ng mga typed message (hal., `RequestResponse[HumanFeedbackRequest, str]`)
- Mag-route ng workflow sa pamamagitan ng pagpapadala ng mga mensahe sa ibang mga executor
- Mag-access ng context sa pamamagitan ng `WorkflowContext`

#### 5. **Conditional Routing gamit ang Mga Desisyon ng Tao**
Maaari kang gumawa ng mga function na nag-evaluate sa mga sagot ng tao:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Mga Aplikasyon sa Tunay na Mundo:

1. **Approval Workflows**
   - Kunin ang pag-apruba ng manager bago iproseso ang mga expense report
   - Kailangan ng human review bago magpadala ng automated na email
   - Kumpirmahin ang mga transaksyong may mataas na halaga bago ito isagawa

2. **Content Moderation**
   - I-flag ang mga kahina-hinalang content para sa human review
   - Tanungin ang mga moderator na gumawa ng huling desisyon sa mga edge case
   - I-escalate sa tao kapag mababa ang kumpiyansa ng AI

3. **Customer Service**
   - Hayaan ang AI na awtomatikong sagutin ang mga karaniwang tanong
   - I-escalate ang mga komplikadong isyu sa mga human agent
   - Tanungin ang customer kung gusto nilang makipag-usap sa tao

4. **Data Processing**
   - Tanungin ang tao upang lutasin ang mga hindi malinaw na data entry
   - Kumpirmahin ang interpretasyon ng AI sa mga hindi malinaw na dokumento
   - Hayaan ang user na pumili sa pagitan ng maraming valid na interpretasyon

5. **Safety-Critical Systems**
   - Kailangan ng kumpirmasyon ng tao bago ang mga hindi maibabalik na aksyon
   - Kunin ang pag-apruba bago ma-access ang sensitibong data
   - Kumpirmahin ang mga desisyon sa mga regulated na industriya (healthcare, finance)

6. **Interactive Agents**
   - Gumawa ng mga conversational bot na nagtatanong ng mga follow-up na tanong
   - Lumikha ng mga wizard na gumagabay sa mga user sa mga kumplikadong proseso
   - Magdisenyo ng mga agent na nakikipagtulungan sa tao hakbang-hakbang

### 🔄 Paghahambing: May vs Walang Human-in-the-Loop

| Katangian | Conditional Workflow | Human-in-the-Loop Workflow |
|-----------|-----------------------|---------------------------|
| **Execution** | Single `workflow.run()` | Loop gamit ang `run_stream()` + `send_responses_streaming()` |
| **User Input** | Wala (fully automated) | Interactive prompts gamit ang `input()` o UI |
| **Components** | Agents + Executors | + RequestInfoExecutor + DecisionManager |
| **Events** | AgentExecutorResponse lamang | RequestInfoEvent, WorkflowOutputEvent, atbp. |
| **Pausing** | Walang pagpigil | Workflow ay pinipigil sa RequestInfoExecutor |
| **Human Control** | Walang kontrol ng tao | Ang tao ang gumagawa ng mahahalagang desisyon |
| **Use Case** | Automation | Pakikipagtulungan at oversight |

### 🚀 Mga Advanced na Pattern:

#### Maramihang Human Decision Points
Maaari kang magkaroon ng maraming `RequestInfoExecutor` nodes sa parehong workflow:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Timeout Handling
Magpatupad ng timeout para sa mga sagot ng tao:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Rich UI Integration
Sa halip na `input()`, mag-integrate gamit ang web UI, Slack, Teams, atbp.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Conditional Human-in-the-Loop
Humingi lamang ng input ng tao sa mga partikular na sitwasyon:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Mga Pinakamahusay na Praktika:

1. **Laging I-subclass ang RequestInfoMessage**
   - Nagbibigay ng type safety at validation
   - Nagbibigay ng rich context para sa UI rendering
   - Nililinaw ang layunin ng bawat uri ng request

2. **Gumamit ng Descriptive Prompts**
   - Isama ang konteksto tungkol sa kung ano ang tinatanong
   - Ipaliwanag ang mga kahihinatnan ng bawat pagpipilian
   - Panatilihing simple at malinaw ang mga tanong

3. **I-handle ang Hindi Inaasahang Input**
   - I-validate ang mga sagot ng user
   - Magbigay ng default para sa invalid na input
   - Magbigay ng malinaw na error message

4. **Subaybayan ang Request IDs**
   - Gamitin ang correlation sa pagitan ng request_id at mga sagot
   - Huwag subukang i-manage ang estado nang manu-mano

5. **Magdisenyo para sa Non-Blocking**
   - Huwag i-block ang mga thread habang naghihintay ng input
   - Gumamit ng async patterns sa kabuuan
   - Suportahan ang sabay-sabay na workflow instances

### 📚 Mga Kaugnay na Konsepto:

- **Agent Middleware** - I-intercept ang mga tawag ng agent (nakaraang notebook)
- **Workflow State Management** - I-persist ang estado ng workflow sa pagitan ng mga run
- **Multi-Agent Collaboration** - Pagsamahin ang human-in-the-loop sa mga team ng agent
- **Event-Driven Architectures** - Gumawa ng reactive systems gamit ang mga event

---

### 🎓 Binabati Kita!

Na-master mo na ang human-in-the-loop workflows gamit ang Microsoft Agent Framework! Alam mo na kung paano:
- ✅ Pigilan ang workflows upang mangolekta ng input ng tao
- ✅ Gumamit ng RequestInfoExecutor at RequestInfoMessage
- ✅ I-handle ang streaming execution gamit ang mga event
- ✅ Gumawa ng custom executors gamit ang @handler
- ✅ Mag-route ng workflows batay sa mga desisyon ng tao
- ✅ Bumuo ng interactive AI agents na nakikipagtulungan sa tao

**Ito ay isang mahalagang pattern para sa paggawa ng mapagkakatiwalaan at kontroladong AI systems!** 🚀



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, mangyaring tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na mapagkakatiwalaang pinagmulan. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
